In [1]:
import numpy as np
np.set_printoptions(suppress=True, precision=3, linewidth=250)
from websocket_server import WebsocketServer
import meshio
import json

In [2]:
server = WebsocketServer(host='127.0.0.1', port=5678)
server.run_forever(threaded=True)

In [58]:
class Mesh:
    def __init__(self, path, scale, color):
        self.data = mesh = meshio.read(path)        
        self.scale = scale
        self.color = color

        # Frontend expects flattened list.
        self.vertices = np.array(mesh.points).reshape(-1)  
        self.indices = np.array(mesh.cells_dict['triangle']).reshape(-1)

In [65]:
sphere = Mesh('sphere.obj', [0.01, 0.01, 0.01], '0x007bff')

In [63]:
def msgMesh(name, mesh):
    return {
        '__static__': True,
        'name': name,
        'data': {
            'name': name,
            'type': '3dMesh',
            'vertices': mesh.vertices,
            'indices': mesh.indices,
            'scale': mesh.scale,
            'color': mesh.color
        }
    }

In [102]:
def json_default(obj):
    if type(obj).__module__ == np.__name__:
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj.item()
    raise TypeError('Unknown type:', type(obj))

def sendMsg(data):
    server.send_message_to_all(json.dumps(data, default=json_default))

sendMsg([
    {
        'time': 0.0,
        '3d/sphere/pos': [0., 0, 0.1, 0, 0, 0]
    }, 
    msgMesh('3d/sphere', sphere),
    msgMesh('3d/sphere2', sphere)
])

for ti in range(1000):
    t = 0.001 * ti
    sendMsg([{
        'time': t,
        '3d/sphere/pos': [0.1 * np.sin(2 * np.pi * t), 0.2 * np.cos(2 * np.pi * t), 0.1, 0, 0, 0]
    }])

In [98]:
np.pi

3.141592653589793